In [1]:
import numpy as np
import pandas as pd

from functools import partial

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

Загрузим датасет с жильем в Бостоне, разделим выборки на обучающую и тестовую, выполним стандартизацию.

In [2]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

## 1. Тестирование модели SVR

Для каждой тестируемой модели будем выполнять байесовскую оптимизацию и случайный решетчатый поиск.

Для байесовской оптимизации создадим функцию кросс-валидации.

In [3]:
def cross_validation(params, estimator, X, y, cv=2):
    """
    Кросс-валидация с текущими гиперпараметрами
    
    params : dict
      Словарь гиперпараметров
    estimator : sklearn model
      Модель sklearn
    X : matrix
      Матрица признаков
    y : вектор целевых значений   
    """
    
    estimator.set_params(**params)
    
    score = cross_val_score(estimator=estimator, X=X, y=y, cv=cv)
    
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

In [4]:
search_space = hp.choice(label='params', options=[
    
    {'kernel': hp.choice(label='criterion_1',  options=['linear']),
     'C': hp.uniform(label='C_1', low=1e-12, high=10)},
    
    {'kernel': hp.choice(label='criterion_2',  options=['sigmoid']),
     'C': hp.uniform(label='C_2', low=1e-12, high=10),
     'coef0': hp.uniform(label='coef0_2', low=1e-12, high=10)},
    
    {'kernel': hp.choice(label='criterion_3',  options=['rbf']),
     'C': hp.uniform(label='C_3', low=1e-12, high=10)},
    
    {'kernel': hp.choice(label='criterion_4',  options=['poly']),
     'degree': hp.quniform(label='degree', low=3, high=8, q=1),
     'C': hp.uniform(label='C_4', low=1e-12, high=2),
     'coef0': hp.uniform(label='coef0_4', low=1e-12, high=10)},
]
)

In [5]:
trials = Trials()
svr_model = SVR()
bayesian_search = fmin( 
  # функция для оптимизации  
    fn=partial(cross_validation, estimator=svr_model, X=X_train_std, y=y_train, cv=5),
  # пространство поиска гиперпараметров  
    space=search_space,
  # алгоритм поиска
    algo=tpe.suggest,
  # число итераций 
  # (можно ещё указать и время поиска) 
    max_evals=50,
  # куда сохранять историю поиска
    trials=trials,
  # random state
    rstate=np.random.RandomState(42),
  # progressbar
    show_progressbar=True
)

100%|███████████████████████████████████████████████| 50/50 [02:35<00:00,  3.11s/trial, best loss: -0.8259198885966708]


In [6]:
print('SVR, байесовская оптимизация\n')
print('Лучшая метрика R2:', -trials.best_trial['result']['loss'])
print('Лучшие параматры:\n', bayesian_search, sep='')

SVR, байесовская оптимизация

Лучшая метрика R2: 0.8259198885966708
Лучшие параматры:
{'C_4': 0.9146782376276084, 'coef0_4': 8.118629376481968, 'criterion_4': 0, 'degree': 3.0, 'params': 3}


В результате байесовской оптимизации params = 3 и criterion_4 = 0 означают, что было выбрано полиномиальное ядро.

Теперь проверим какие результаты даст RandomizedSearchCV с аналогичными параметрами поиска.

In [7]:
params = [
    {'kernel': ['linear'],
     'C': np.linspace(1e-12, 10, 1000)},
    
    {'kernel': ['sigmoid'],
     'C': np.linspace(1e-12, 10, 1000),
     'coef0': np.linspace(1e-12, 10, 1000)},
    
    {'kernel': ['rbf'],
     'C': np.linspace(1e-12, 10, 1000)},
    
    {'kernel': ['poly'],
     'degree': np.arange(2,9,1),
     'C': np.linspace(1e-12, 10, 1000),
     'coef0': np.linspace(1e-12, 10, 1000)},
]

In [8]:
rand_search = RandomizedSearchCV(estimator=svr_model, param_distributions=params, cv=5, n_iter=50,
                                 random_state=42)

In [9]:
# Подгонка может идти 10 минут и более
rand_search.fit(X_train_std, y_train)

RandomizedSearchCV(cv=5,
                   estimator=SVR(C=7.861927986590262, coef0=7.241456456254678,
                                 degree=4.0),
                   n_iter=50,
                   param_distributions=[{'C': array([1.00000000e-12, 1.00100100e-02, 2.00200200e-02, 3.00300300e-02,
       4.00400400e-02, 5.00500501e-02, 6.00600601e-02, 7.00700701e-02,
       8.00800801e-02, 9.00900901e-02, 1.00100100e-01, 1.10110110e-01,
       1.20120120e-01, 1.30130130e-01, 1.4...
       9.80980981e+00, 9.81981982e+00, 9.82982983e+00, 9.83983984e+00,
       9.84984985e+00, 9.85985986e+00, 9.86986987e+00, 9.87987988e+00,
       9.88988989e+00, 9.89989990e+00, 9.90990991e+00, 9.91991992e+00,
       9.92992993e+00, 9.93993994e+00, 9.94994995e+00, 9.95995996e+00,
       9.96996997e+00, 9.97997998e+00, 9.98998999e+00, 1.00000000e+01]),
                                         'degree': array([2, 3, 4, 5, 6, 7, 8]),
                                         'kernel': ['poly']}],
              

In [10]:
print('SVR, случайный решетчатый поиск\n')
print('Лучшая метрика R2:', rand_search.best_score_)
print('Лучшие параматры:\n', rand_search.best_params_, sep='')

SVR, случайный решетчатый поиск

Лучшая метрика R2: 0.830476316410422
Лучшие параматры:
{'kernel': 'poly', 'degree': 2, 'coef0': 4.784784784785306, 'C': 8.98898898898909}


Случайный решетчатый поиск сработал немного лучше.

## 2. Тестирование модели дерева принятий решений

In [11]:
dectree_model = DecisionTreeRegressor()

In [12]:
search_space = {'criterion': hp.choice(label='criterion',  options=["mse", "friedman_mse", "mae", "poisson"]),
                'splitter': hp.choice(label='splitter',  options=["best", "random"]),
                'max_depth': hp.quniform(label='max_depth', low=1, high=1000, q=1),
                'random_state': hp.choice(label='random_state',  options=[42])}

trials = Trials()
bayesian_search = fmin( 
    fn=partial(cross_validation, estimator=dectree_model, X=X_train_std, y=y_train, cv=5),
    space=search_space,
    algo=tpe.suggest,
    max_evals=500,
    trials=trials,
    rstate=np.random.RandomState(42),
    show_progressbar=True
)

100%|██████████████████████████████████████████████| 500/500 [00:21<00:00, 23.20trial/s, best loss: -0.732071724762168]


In [13]:
print('Дерево принятий решений, байесовская оптимизация\n')
print('Лучшая метрика R2:', -trials.best_trial['result']['loss'])
print('Лучшие параматры:\n', bayesian_search, sep='')

Дерево принятий решений, байесовская оптимизация

Лучшая метрика R2: 0.732071724762168
Лучшие параматры:
{'criterion': 0, 'max_depth': 18.0, 'random_state': 0, 'splitter': 0}


В данном случае criterion = 0 обозначает criterion = mse, а splitter = 0 обозначает splitter = best.

Теперь проверим какие результаты даст RandomizedSearchCV с аналогичными параметрами поиска.

In [14]:
params = {'criterion': ["mse", "friedman_mse", "mae", "poisson"],
          'splitter': ["best", "random"],
          'max_depth': range(1, 1000),
          'random_state': [42]}

rand_search = RandomizedSearchCV(estimator=dectree_model, param_distributions=params, cv=5, n_iter=500,
                                 random_state=42)
rand_search.fit(X_train_std, y_train)

print('Дерево принятий решений, cлучайный решетчатый поиск\n')
print('Лучшая метрика R2:', rand_search.best_score_)
print('Лучшие параматры:\n', rand_search.best_params_, sep='')

Дерево принятий решений, cлучайный решетчатый поиск

Лучшая метрика R2: 0.7308587061213911
Лучшие параматры:
{'splitter': 'best', 'random_state': 42, 'max_depth': 354, 'criterion': 'mse'}


В этот раз байесовская оптимизация сработала немного лучше.

## 3. Тестирование модели случайного леса деревьев

Под случайный лес необходимо переделать функцию кросс-валидации, у библиотеки hyperopt не нашел возможности задать количество n_estimators в целых числах с исключением нуля. Только целые числа в формате float через hp.quniform с q = 1.

In [15]:
def cross_validation_randforest(params, estimator, X, y, cv=2):
    params['n_estimators'] = int(params['n_estimators'])
    estimator.set_params(**params)
    score = cross_val_score(estimator=estimator, X=X, y=y, cv=cv)
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}


randree_model = RandomForestRegressor()

search_space = {'criterion': hp.choice(label='criterion',  options=["mse", "mae"]),
                'n_estimators': hp.quniform(label='n_estimators', low=10, high=200, q=1),
                'random_state': hp.choice(label='random_state',  options=[42])}

trials = Trials()
bayesian_search = fmin( 
    fn=partial(cross_validation_randforest, estimator=randree_model, X=X_train_std, y=y_train, cv=5),
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.RandomState(42),
    show_progressbar=True
)

100%|███████████████████████████████████████████████| 50/50 [01:41<00:00,  2.04s/trial, best loss: -0.8140513014053363]


In [16]:
print('Случайный лес, байесовская оптимизация\n')
print('Лучшая метрика R2:', -trials.best_trial['result']['loss'])
print('Лучшие параматры:\n', bayesian_search, sep='')

Случайный лес, байесовская оптимизация

Лучшая метрика R2: 0.8140513014053363
Лучшие параматры:
{'criterion': 0, 'n_estimators': 113.0, 'random_state': 0}


In [17]:
params = {'criterion': ["mse", "mae"],
          'n_estimators': range(10, 200),
          'random_state': [42]}

rand_search = RandomizedSearchCV(estimator=randree_model, param_distributions=params, cv=5, n_iter=50,
                                 random_state=42)
rand_search.fit(X_train_std, y_train)

print('Случайный лес, cлучайный решетчатый поиск\n')
print('Лучшая метрика R2:', rand_search.best_score_)
print('Лучшие параматры:\n', rand_search.best_params_, sep='')

Случайный лес, cлучайный решетчатый поиск

Лучшая метрика R2: 0.8137525844097379
Лучшие параматры:
{'random_state': 42, 'n_estimators': 136, 'criterion': 'mse'}


Для случайного леса байесовская оптимизация дала лучший результат, но незначительно.

## 4. Тестирование модели BayesianRidge

In [18]:
bayesianridge_model = BayesianRidge()

In [19]:
search_space = {'alpha_1': hp.uniform(label='alpha_1', low=1e-12, high=1e-3),
                'alpha_2': hp.uniform(label='alpha_2', low=1e-12, high=1e-3),
                'lambda_1': hp.uniform(label='lambda_1', low=1e-12, high=1e-3),
                'lambda_2': hp.uniform(label='lambda_2', low=1e-12, high=1e-3)}

trials = Trials()
bayesian_search = fmin( 
    fn=partial(cross_validation, estimator=bayesianridge_model, X=X_train_std, y=y_train, cv=5),
    space=search_space,
    algo=tpe.suggest,
    max_evals=500,
    trials=trials,
    rstate=np.random.RandomState(42),
    show_progressbar=True
)

100%|██████████████████████████████████████████████| 500/500 [00:11<00:00, 43.47trial/s, best loss: -0.690135701077619]


In [20]:
print('BayesianRidge, байесовская оптимизация\n')
print('Лучшая метрика R2:', -trials.best_trial['result']['loss'])
print('Лучшие параматры:\n', bayesian_search, sep='')

BayesianRidge, байесовская оптимизация

Лучшая метрика R2: 0.690135701077619
Лучшие параматры:
{'alpha_1': 2.207406862020356e-05, 'alpha_2': 0.00012170803513291944, 'lambda_1': 0.0009989520424490438, 'lambda_2': 1.438476970661669e-06}


In [21]:
params = {'alpha_1': np.linspace(1e-12, 1e-1, 100),
          'alpha_2': np.linspace(1e-12, 1e-1, 100),
          'lambda_1': np.linspace(1e-12, 1e-1, 100),
          'lambda_2': np.linspace(1e-12, 1e-1, 100)}

rand_search = RandomizedSearchCV(estimator=bayesianridge_model, param_distributions=params, cv=5, n_iter=50,
                                 random_state=42)
rand_search.fit(X_train_std, y_train)

print('BayesianRidge, случайный решетчатый поиск\n')
print('Лучшая метрика R2:', rand_search.best_score_)
print('Лучшие параматры:\n', rand_search.best_params_, sep='')

BayesianRidge, случайный решетчатый поиск

Лучшая метрика R2: 0.6901564790022852
Лучшие параматры:
{'lambda_2': 0.003030303031272727, 'lambda_1': 0.09090909090918181, 'alpha_2': 0.0040404040413636365, 'alpha_1': 0.03232323232390909}


## 5. Тестирование ансамблевой модели BaggingRegressor на основе деревьев принятий решений

In [22]:
bagging = BaggingRegressor()

search_space = {'max_samples': hp.uniform(label='max_samples', low=0.1, high=1),
                'max_features': hp.uniform(label='max_features ', low=0.1, high=1),
                'n_estimators': hp.quniform(label='n_estimators', low=5, high=30, q=1),
                'random_state': hp.choice(label='random_state',  options=[42])}

trials = Trials()
bayesian_search = fmin( 
    fn=partial(cross_validation_randforest, estimator=bagging, X=X_train_std, y=y_train, cv=5),
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.RandomState(42),
    show_progressbar=True
)

100%|███████████████████████████████████████████████| 50/50 [00:11<00:00,  4.51trial/s, best loss: -0.8146614016618127]


In [23]:
print('Бэггинг из деревьев решений, байесовская оптимизация\n')
print('Лучшая метрика R2:', -trials.best_trial['result']['loss'])
print('Лучшие параматры:\n', bayesian_search, sep='')

Бэггинг из деревьев решений, байесовская оптимизация

Лучшая метрика R2: 0.8146614016618127
Лучшие параматры:
{'max_features ': 0.9737584269557823, 'max_samples': 0.7655632847531592, 'n_estimators': 30.0, 'random_state': 0}


In [24]:
params = {'max_samples': np.linspace(0.1, 1, 100),
          'max_features': np.linspace(0.1, 1, 100),
          'n_estimators': range(5, 30),
          'random_state': [42]}

rand_search = RandomizedSearchCV(estimator=bagging, param_distributions=params, cv=5, n_iter=50,
                                 random_state=42)
rand_search.fit(X_train_std, y_train)

print('Бэггинг из деревьев решений, случайный решетчатый поиск\n')
print('Лучшая метрика R2:', rand_search.best_score_)
print('Лучшие параматры:\n', rand_search.best_params_, sep='')

Бэггинг из деревьев решений, случайный решетчатый поиск

Лучшая метрика R2: 0.7982897005130849
Лучшие параматры:
{'random_state': 42, 'n_estimators': 6, 'max_samples': 0.709090909090909, 'max_features': 0.8727272727272727}


В этом тесте байесовская оптимизация дала более лучший результат.